In [ ]:
cd ..

In [ ]:
import glob
import pickle as pkl
import h5py

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from sparse_coder.sparse_coder import SparseCoder
from utils.rf_plot import show_fields
from sparse_coder.prep_field_dataset import get_data_matrix

In [ ]:
files = glob.glob('sparse_coder/output/vh_sparse_coder_alpha_*_overcomp_*.pkl')
files = sorted(files)

files

In [ ]:
path = files[1]
path

In [ ]:
with open(path, 'r') as f:
    out = pkl.load(f)
    D = out['D']
    print out['alpha']

In [ ]:
idx = np.random.randint(D.shape[0], size=512)

fig, ax = plt.subplots(1, 1, figsize=(16, 14))
show_fields(D, fig=fig, ax=ax)
ax.set_title('Dictionary with alpha={:.2f} n_sp = {}'.format(out['alpha'], out['n_sp']))
# plt.savefig('sparse_coder/output/dict.pdf', dpi=300)

In [ ]:
with h5py.File('sparse_coder/data/final/new_extracted_patches.h5') as f:
    data = f['white_patches'][0:1000]
    data = data.reshape(1000, -1)

In [ ]:
sc = SparseCoder.restore(data, path)

In [ ]:
cost_list = []
i_idx = sc.train(n_itr=1, eta=0, cost_list=cost_list, n_g_itr=100)

In [ ]:
sc.plot_example(3, i_idx, 32)

In [ ]:
pcts = np.percentile(abs(data.ravel()), 99)

In [ ]:
idx_ = np.where(abs(data) > 1)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
q = 400
cax = axes[0].imshow(data[q].reshape(20, 20), cmap=plt.cm.gray, interpolation='nearest')
fig.colorbar(cax, ax=axes[0])
cax = axes[1].imshow(np.clip(data[q], -1, 1).reshape(20, 20), 
                     cmap=plt.cm.gray, interpolation='nearest', vmin=-1, vmax=1)
fig.colorbar(cax, ax=axes[1])


In [ ]:
_ = plt.hist(abs(data.ravel()), bins=50)
plt.yscale('log')

In [ ]:
def normalize(x, smin, smax):
    xmin, xmax = [getattr(x, func)(axis=1, keepdims=True) for func in ['min', 'max']]
    u = (x - xmin) / (xmax-xmin)
    return u * (smax - smin) + smin

In [ ]:
data_normed = normalize(data, -0.5, 0.5)

In [ ]:
_ = plt.hist(data_normed.ravel(), bins=50)

In [ ]:
np.random.randint(0, 100, size=10)

In [ ]:
plt.hist(data_normed.std(axis=1))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
q = 10
cax = axes[0].imshow(data[q].reshape(20, 20), cmap=plt.cm.gray, interpolation='nearest')
fig.colorbar(cax, ax=axes[0])
cax = axes[1].imshow(np.clip(data_normed[q], -1, 1).reshape(20, 20), 
                     cmap=plt.cm.gray, interpolation='nearest', vmin=-0.5, vmax=0.5)
fig.colorbar(cax, ax=axes[1])


In [ ]:
plt.hist(data.mean(axis=1), bins=50)